In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "f5d6f843",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/javascript": [
       "\n",
       "try {\n",
       "require(['notebook/js/codecell'], function(codecell) {\n",
       "  codecell.CodeCell.options_default.highlight_modes[\n",
       "      'magic_text/x-csrc'] = {'reg':[/^%%microblaze/]};\n",
       "  Jupyter.notebook.events.one('kernel_ready.Kernel', function(){\n",
       "      Jupyter.notebook.get_cells().map(function(cell){\n",
       "          if (cell.cell_type == 'code'){ cell.auto_highlight(); } }) ;\n",
       "  });\n",
       "});\n",
       "} catch (e) {};\n"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "application/javascript": [
       "\n",
       "try {\n",
       "require(['notebook/js/codecell'], function(codecell) {\n",
       "  codecell.CodeCell.options_default.highlight_modes[\n",
       "      'magic_text/x-csrc'] = {'reg':[/^%%pybind11/]};\n",
       "  Jupyter.notebook.events.one('kernel_ready.Kernel', function(){\n",
       "      Jupyter.notebook.get_cells().map(function(cell){\n",
       "          if (cell.cell_type == 'code'){ cell.auto_highlight(); } }) ;\n",
       "  });\n",
       "});\n",
       "} catch (e) {};\n"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "from pynq import Overlay\n",
    "import time\n",
    "import matplotlib.pyplot as plt\n",
    "import numpy as np\n",
    "\n",
    "ol = Overlay(\"/home/xilinx/please_work/working.bit\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "9b263a02",
   "metadata": {},
   "outputs": [],
   "source": [
    "oux = ol.u_x.channel1\n",
    "ouy = ol.u_y.channel1\n",
    "orho = ol.rho.channel1\n",
    "istep = ol.step.channel1\n",
    "ireset_lbm = ol.reset_lbm.channel1\n",
    "istart_transmit = ol.start_transmit.channel1\n",
    "ostep_countn = ol.step_countn.channel1\n",
    "iparam_set = ol.params.channel1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "4502fa0f",
   "metadata": {},
   "outputs": [],
   "source": [
    "def write_param(param, val):\n",
    "    # Ensure value fits in 16 bits\n",
    "    val_16bit = val & 0xFFFF  \n",
    "    \n",
    "    # Create 32-bit word: [16-bit index | 16-bit value]\n",
    "    if param == \"omega\":\n",
    "        gpio_value = (0 << 16) | val_16bit\n",
    "    elif param == \"c0\":\n",
    "        gpio_value = (1 << 16) | val_16bit\n",
    "    elif param == \"cn\":\n",
    "        gpio_value = (2 << 16) | val_16bit\n",
    "    elif param == \"cne\":\n",
    "        gpio_value = (3 << 16) | val_16bit\n",
    "    elif param == \"ce\":\n",
    "        gpio_value = (4 << 16) | val_16bit\n",
    "    elif param == \"cse\":\n",
    "        gpio_value = (5 << 16) | val_16bit\n",
    "    elif param == \"cs\":\n",
    "        gpio_value = (6 << 16) | val_16bit\n",
    "    elif param == \"csw\":\n",
    "        gpio_value = (7 << 16) | val_16bit\n",
    "    elif param == \"cw\":\n",
    "        gpio_value = (8 << 16) | val_16bit\n",
    "    elif param == \"cnw\":\n",
    "        gpio_value = (9 << 16) | val_16bit\n",
    "    else:\n",
    "        raise ValueError(f\"Unknown parameter: {param}\")\n",
    "    \n",
    "    iparam_set.write(gpio_value,-1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "bb60c3fe",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "reset state: 0\n",
      "reset state: 1\n",
      "transmit state: 0\n",
      "current step (expect 0): 0\n"
     ]
    }
   ],
   "source": [
    "# set constants (all 0)\n",
    "# write_param(\"omega\", 0x3c61)\n",
    "# write_param(\"c0\",  0x0381)   # n0[θ] - Center\n",
    "# write_param(\"cn\",  0x0381)   # nN[θ] - North\n",
    "# write_param(\"cs\",  0x0381)   # nS[θ] - South \n",
    "# write_param(\"ce\",  0x04BB)   # nE[θ] - East\n",
    "# write_param(\"cw\",  0x0298)   # nW[θ] - West\n",
    "# write_param(\"cne\", 0x012F)   # nNE[θ] - Northeast\n",
    "# write_param(\"cse\", 0x012F)   # nSE[θ] - Southeast\n",
    "# write_param(\"cnw\", 0x00A6)   # nNW[θ] - Northwest\n",
    "# write_param(\"csw\", 0x00A6)   # nSW[θ] - Southwest\n",
    "\n",
    "# set constants (eastward)\n",
    "write_param(\"omega\", 0x3c61)\n",
    "write_param(\"c0\",  0x2000)   # n0[θ] - Center\n",
    "write_param(\"cn\",  0x2000)   # nN[θ] - North\n",
    "write_param(\"cs\",  0x2000)   # nS[θ] - South \n",
    "write_param(\"ce\",  0x599a)   # nE[θ] - East\n",
    "write_param(\"cw\",  0x2000)   # nW[θ] - West\n",
    "write_param(\"cne\", 0x2000)   # nNE[θ] - Northeast\n",
    "write_param(\"cse\", 0x2000)   # nSE[θ] - Southeast\n",
    "write_param(\"cnw\", 0x2000)   # nNW[θ] - Northwest\n",
    "write_param(\"csw\", 0x2000)   # nSW[θ] - Southwest\n",
    "\n",
    "# reset all\n",
    "ireset_lbm.write(0,1) #reset\n",
    "print(f\"reset state: {ireset_lbm.read()}\") # check reset\n",
    "time.sleep(1) # wait\n",
    "ireset_lbm.write(1,1) #reset\n",
    "print(f\"reset state: {ireset_lbm.read()}\") # check non-reset\n",
    "\n",
    "# check start transmit\n",
    "istart_transmit.write(0,-1)\n",
    "print(f\"transmit state: {istart_transmit.read()}\")\n",
    "istep.write(0,-1)\n",
    "time.sleep(1)\n",
    "# check step\n",
    "print(f\"current step (expect 0): {ostep_countn.read()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "b55682da",
   "metadata": {
    "scrolled": true
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "done\n"
     ]
    }
   ],
   "source": [
    "for step in range(0,100):\n",
    "    istep.write(step,0x00FF)\n",
    "#     time.sleep(0.01)\n",
    "    rhos = []\n",
    "    \n",
    "#     print(f\"solver current step : {ostep_countn.read()}\")\n",
    "\n",
    "    istart_transmit.write(0x8000,0x8000) # raise query state flag\n",
    "    for pixel in range(2500):\n",
    "        istart_transmit.write(pixel,0x0FFF) # get pixel\n",
    "        rhos.append(float(orho.read())/(2**13))\n",
    "    plt.imsave(f\"imgs/step{step}.png\", np.reshape(rhos,(50,50)), vmin=-4, vmax=4)\n",
    "\n",
    "\n",
    "    istart_transmit.write(0,0x8000)\n",
    "print(\"done\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8b9ea0b2",
   "metadata": {},
   "outputs": [],
   "source": [
    "with open(\"binout_lil.bytes\", \"wb\") as f:\n",
    "    for step in range(0,100):\n",
    "        istep.write(step,0x00FF)\n",
    "#         print(f\"solver current step : {ostep_countn.read()}\")\n",
    "        istart_transmit.write(0x8000,0x8000) # raise query state flag\n",
    "        for pixel in range(2500):\n",
    "            istart_transmit.write(pixel,0x0FFF) # get pixel\n",
    "            for val in [orho.read(), oux.read(), ouy.read()]:\n",
    "                f.write((val).to_bytes(2, byteorder=\"little\"))\n",
    "        istart_transmit.write(0,0x8000) # lower query state flag"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "8edf62ac",
   "metadata": {},
   "source": [
    "# Utility"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ff5353dd",
   "metadata": {},
   "outputs": [],
   "source": [
    "# clear images\n",
    "import os\n",
    "os.system(\"rm imgs/*\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "75cb94fb",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'/home/xilinx/jupyter_notebooks/fixed_please.zip'"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "import shutil\n",
    "shutil.make_archive(\"fixed_please\", 'zip', \"imgs\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f7b40d0f",
   "metadata": {},
   "source": [
    "# unused tests"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "720352a2",
   "metadata": {},
   "outputs": [],
   "source": [
    "# test to see if there are any changes in outputs\n",
    "prev_rhos = []\n",
    "for step in range(0,100):\n",
    "#     iinterface_reset.write(0,1)\n",
    "    istep.write(step,0x00FF)\n",
    "\n",
    "    rhos = []\n",
    "\n",
    "    istart_transmit.write(0x8000,0x8000) # raise query state flag\n",
    "    for pixel in range(2500):\n",
    "        istart_transmit.write(pixel,0x0FFF) # get pixel\n",
    "        rhos.append(oux.read())\n",
    "\n",
    "    istart_transmit.write(0,-1)\n",
    "#     iinterface_reset.write(1,1)\n",
    "    \n",
    "    if(prev_rhos == []):\n",
    "        prev_rhos = rhos.copy()\n",
    "    else:\n",
    "        print(f\"solver current step : {ostep_countn.read()}, rhos equal? {prev_rhos == rhos}\")\n",
    "    \n",
    "print(f\"ux at step 1, pixel 70: {oldux.read() & 0xFFFF}, ux at step 20, pixel 70: {oldux.read() >> 13}\") "
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


In [2]:
oux = ol.u_x.channel1
ouy = ol.u_y.channel1
orho = ol.rho.channel1
istep = ol.step.channel1
ireset_lbm = ol.reset_lbm.channel1
istart_transmit = ol.start_transmit.channel1

In [21]:
# reset all
ireset_lbm.write(0,1) #reset
print(f"reset state: {ireset_lbm.read()}") # check reset
time.sleep(1) # wait
ireset_lbm.write(1,1) #reset
print(f"reset state: {ireset_lbm.read()}") # check non-reset

# check start transmit
istart_transmit.write(0,-1)
print(f"transmit state: {istart_transmit.read()}")

reset state: 0
reset state: 1
transmit state: 0


In [22]:
for step in range(0,1000,100):
    istep.write(step,0xFFFF)
#     print(f"current step: {istep.read()}")
    time.sleep(0.1) # give it way more time than it needs
    uxs = []
    rhos = []
    istart_transmit.write(0x8000,0x8000) # raise query state flag
    for pixel in range(2500):
        istart_transmit.write(pixel,0x0FFF) # get pixel
#             print(f"transmit state: {istart_transmit.read()}")
        rhos.append(float(orho.read())/(2**13))
#             time.sleep(0.0005) # give it time
    plt.imsave(f"imgs/step{step}.png", np.reshape(rhos,(50,50)))

In [23]:
import shutil
shutil.make_archive("img_zip", 'zip', "imgs")

'/home/xilinx/jupyter_notebooks/img_zip.zip'